# Lyft Prediction Exploration

Description: TODO

## Explore the data folders

Let's fisrt familiarize what are provided in the input folder.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Add Lyft utility script

This is only needed once.

> File -> Add utility script -> Searach with "kaggle-l5kit". (Cancel the filter if there is any. In my case, I see the filter "My Work" there in default.) -> Pick "Lyft - l5kit (unofficial fix)" and Click "Add". (Tried with "kaggle_l5kit" but always have issuse in saving versions.)

## Attach additional dataset source: for config files

We are using some config files when we want to load/visualize Lyft level5 dataset.<br/>
@jpbremer already uploaded config files as Kaggle Dataset platform: [lyft-config-files](https://www.kaggle.com/jpbremer/lyft-config-files).<br/>
This is originally from official github [lyft/l5kit example page](https://github.com/lyft/l5kit/tree/master/examples).

Click "Add data" button and press "Search by URL". Typing "https://www.kaggle.com/jpbremer/lyft-config-files" shows the dataset.<br/>
Once the dataset is successfully added you can see "lyft-config-files" as dataset on left side bar.

## Explore and prepare the python environment

- Check the kernel
- Import the libararies

In [ ]:
import sys
sys.executable

In [ ]:
#!pip list

In [ ]:
import l5kit
l5kit.__version__

In [ ]:
import torch
torch.__version__

In [ ]:
# check if GPU available
torch.cuda.is_available()

### Import libaries

We don't have to find all the needed libaries in the beginning. As we are completing each step, we can add in the relevant libaries in here.

In [ ]:
# common
import os
import time
import random

# data processing and visualization
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from prettytable import PrettyTable

# torch
import torch

# l5kit
import l5kit
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.data import PERCEPTION_LABELS
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points



from matplotlib import animation, rc
from IPython.display import HTML

rc('animation', html='jshtml')

In [ ]:
# Set the seed to make the results reproducable.
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(42)

## Explore the dataset

### Overview of the dataset

The dataset consists of 170,000 scenes capturing the environment around the autonomous vehicle. Each scene encodes the state of the vehicle’s surroundings at a given point in time.

<p align="center">
<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_lrg_redux.gif" style="width:45%"/>
<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_2-1.png" style="width:45%"/>
</p>

<br/>

<p>
Source: https://level-5.global/data/prediction/
</p>

### Goal

The goal of this competition is to predict other car/cyclist/pedestrian (called "agent")'s motion.

<p align="center">
<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/diagram-prediction-1.jpg" style="width:50%;center"/>
</p>

### numpy structured array

Lyft dataset uses numpy's [structured array](https://docs.scipy.org/doc/numpy/user/basics.rec.html) functionality to store various kinds of features together.<br/>

Let's see example to how it works

In [ ]:
my_arr = np.zeros(3, dtype=[("color", (np.uint8, 3)), ("label", np.bool)])

In [ ]:
my_arr[0]["color"] = [0, 218, 130]
my_arr[0]["label"] = True
my_arr[1]["color"] = [245, 59, 255]
my_arr[1]["label"] = True

In [ ]:
my_arr

**Summary:**

We've defined a length=3 array. Usually each element of array consists of only integer or float, but we can define custom structured format by specifying `dtype` as list of "fields" which consists of name and structure.

Above example contains 2 fields. 1. 8byte uint with length 3 array, 2. single element boolean array.

As you can see, `my_arr[i]["name"]` will access i-th element's "name" field.

Usually when we train neural network, we would like to access all the field of random i-th element.

According to [Data Format page](https://github.com/woven-planet/l5kit/blob/master/docs/data_format.rst), "Structured arrays are a great fit to group this data together in memory and on disk.", rather than preparing "color" array and "label" array separately and access each array's i-th element, especially when the number of field glow. 

### zarr

Zarr data format is used to store and read these numpy structured arrays from disk.

Zarr allows us to write very large (structured) arrays to disk in n-dimensional compressed chunks.

Here is a short tutorial:

In [ ]:
import zarr

z = zarr.open("./dataset.zarr", mode="w", shape=(500,), dtype=np.float32, chunks=(100,))

In [ ]:
# We can write to it by assigning to it. This gets persisted on disk.
z[0:150] = np.arange(150)

In [ ]:
np.arange(10)

**Summary:**

As we specified chunks to be of size 100, we just wrote to two separate chunks. On your filesystem in the dataset.zarr folder you will now find these two chunks. As we didn't completely fill the second chunk, those missing values will be set to the fill value (defaults to 0). The chunks are actually compressed on disk too!

We can print some info: by not doing much work at all we saved almost 75% in disk space!

In [ ]:
print(z.info)

When we check filesystem, `dataset.zarr` directory is created and there are 2 files "0" and "1" which are chunks currently created by just assigning value to zarr array.

In [ ]:
!ls -l ./*

Reading from a zarr array is as easy as slicing from it like you would for any numpy array. The return value is an ordinary numpy array. Zarr takes care of determining which chunks to read from.

In [ ]:
print(z[::20])  # print every 20th element

### Use 'l5kit' to access the data

The dataset structure is a bit complicated since the Level 5 dataset contains various kinds of information. 'l5kit' is provided as a useful library to deal with the data. It is worthywhile to spend a little time to familiarize the the tools we can utilize.

Source: [l5kit visualize_data.ipynb](https://github.com/woven-planet/l5kit/blob/master/examples/visualisation/visualise_data.ipynb).

#### Terminology

 - **"Ego"** is the host car which is recording/measuring the dataset.
 - **"Agent"** is the surronding car except "Ego" car.
 - **"Frame"** is the 1 image snapshot, where **"Scene"** is made of multiple frames of contious-time (video).

#### Class diagram
<img src="https://storage.googleapis.com/kaggle-forum-message-attachments/987047/16744/l5kit_class.png" width="600" />

<br/>

#### Initial setup

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
# get config
cfg = load_config_data("/kaggle/input/lyft-config-files/visualisation_config.yaml")
print(cfg)

### Load the sample data

Here we will only use the first dataset from the sample set.

We're building a `LocalDataManager` object. This will resolve relative paths from the config using the `L5KIT_DATA_FOLDER` env variable we have just set.

Here sample.zarr data is used for visualization, please use train.zarr / validate.zarr / test.zarr for actual model training/validation/prediction.


In [ ]:
dm = LocalDataManager()
dataset_path = dm.require('scenes/sample.zarr')
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

In [ ]:
dataset_path

### Working with the raw data

`zarr_dataset` contains **scenes, frames, agents, tl_faces** attributes, which are the raw structured array data.

Each data structure definition can be checked at [here](https://github.com/lyft/l5kit/blob/master/data_format.md#2020-lyft-competition-dataset-format).

#### scenes

```
SCENE_DTYPE = [
    ("frame_index_interval", np.int64, (2,)),
    ("host", "<U16"),  # Unicode string up to 16 chars
    ("start_time", np.int64),
    ("end_time", np.int64),
]
```

#### frames

```
FRAME_DTYPE = [
    ("timestamp", np.int64),
    ("agent_index_interval", np.int64, (2,)),
    ("traffic_light_faces_index_interval", np.int64, (2,)),
    ("ego_translation", np.float64, (3,)),
    ("ego_rotation", np.float64, (3, 3)),
]
```

#### agents

```
AGENT_DTYPE = [
    ("centroid", np.float64, (2,)),
    ("extent", np.float32, (3,)),
    ("yaw", np.float32),
    ("velocity", np.float32, (2,)),
    ("track_id", np.uint64),
    ("label_probabilities", np.float32, (len(LABELS),)),
]
```

#### traffic_light_faces

```
TL_FACE_DTYPE = [
    ("face_id", "<U16"),
    ("traffic_light_id", "<U16"),
    ("traffic_light_face_status", np.float32, (len(TL_FACE_LABELS,))),
]
```

As an example, we will try scatter plot using **frames "ego_translation"** data. This is the movement of ego car.

In [ ]:
frames = zarr_dataset.frames

## This is slow.
# coords = np.zeros((len(frames), 2))
# for idx_coord, idx_data in enumerate(tqdm(range(len(frames)), desc="getting centroid to plot trajectory")):
#     frame = zarr_dataset.frames[idx_data]
#     coords[idx_coord] = frame["ego_translation"][:2]

# This is much faster!
coords = frames["ego_translation"][:, :2]

plt.scatter(coords[:, 0], coords[:, 1], marker='.')
axes = plt.gca()
axes.set_xlim([-2500, 1600])
axes.set_ylim([-2500, 1600])
plt.title("ego_translation of frames")

**Note:** `frames["ego_translation"]` is same as `frames[:]["ego_translation"]`

### pytorch Dataset class

Instead of working with raw data, L5Kit provides PyTorch ready datasets.
It's much easier to use this wrapped dataset class to access data.

2 dataset class is implemented.

 - **EgoDataset**: this dataset iterates over the AV (Autonomous Vehicle) annotations
 - **AgentDataset**: this dataset iterates over other agents annotations

In [ ]:
# 'map_type': 'py_semantic' for cfg.
# Rasterizer is in charge of visualizing the data, as we will see next.
semantic_rasterizer = build_rasterizer(cfg, dm)
semantic_dataset = EgoDataset(cfg, zarr_dataset, semantic_rasterizer)

## Visualization example

Lyft l5kit also provides visualization functionalities.

We will visualize data to understand what kind of information is stored in this dataset.

Bugfix: https://github.com/woven-planet/l5kit/issues/266

In [ ]:
def visualize_trajectory(dataset, index, title="target_positions movement with draw_trajectory"):
    data = dataset[index]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])

    plt.title(title)
    plt.imshow(im[::-1])
    plt.show()

visualize_trajectory(semantic_dataset, index=0)

We can switch rasterizer to visualize satellite image easily!

In [ ]:
# map_type was changed from 'py_semantic' to 'py_satellite'.
cfg["raster_params"]["map_type"] = "py_satellite"
satellite_rasterizer = build_rasterizer(cfg, dm)
satellite_dataset = EgoDataset(cfg, zarr_dataset, satellite_rasterizer)

visualize_trajectory(satellite_dataset, index=0)

In [ ]:
type(satellite_rasterizer), type(semantic_rasterizer)

Now we visualized **EgoDataset**.

**AgentDataset** can be used to visualize an agent. This dataset iterates over agents and not the AV anymore, and the first one happens to be the pace car (you will see this one around a lot in the dataset).

In [ ]:
agent_dataset = AgentDataset(cfg, zarr_dataset, satellite_rasterizer)
visualize_trajectory(agent_dataset, index=0)

**System Origin and Orientation**

At this point you may have noticed that we flip the image on the Y-axis before plotting it.

When moving from 3D to 2D we stick to a right-hand system, where the origin is in the bottom-left corner with positive x-values going right and positive y-values going up the image plane. The camera is facing down the negative z axis.

However, both opencv and pyplot place the origin in the top-left corner with positive x going right and positive y going down in the image plane. The camera is facing down the positive z-axis.

The flip done on the resulting image is for visualisation purposes to accommodate the difference in the two coordinate frames.

Further, all our rotations are counter-clockwise for positive value of the angle.

## Scene handling

Both EgoDataset and AgentDataset provide 2 methods for getting interesting indices:

 - **get_frame_indices** returns the indices for a given frame. For the `EgoDataset` this matches a single observation, while more than one index could be available for the `AgentDataset`, as that given frame may contain more than one valid agent
 - **get_scene_indices** returns indices for a given scene. For both datasets, these might return more than one index

In [ ]:
from IPython.display import display, clear_output
import PIL
 
dataset = semantic_dataset
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))

In [ ]:
%%capture
# From https://www.kaggle.com/jpbremer/lyft-scene-visualisations by @jpbremer
def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
        return (im,)
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    def init():
        im.set_data(images[0])
        return (im,)
    
    return animation.FuncAnimation(fig, animate, init_func=init, frames=len(images), interval=60, blit=True)

anim = animate_solution(images)

In [ ]:
HTML(anim.to_jshtml())